In [ ]:
import os, sys, email
import numpy as np
import pandas as pd

In [124]:
from email.parser import Parser

path = "enron/maildir/allen-p/inbox/5."

with open(path) as f:
    mail = Parser().parse(f)

In [64]:
cols = ['Path', 'Message-ID', 'Date', 'From', 'To', 'Cc', 'Bcc', 'X-From', 'X-To', 'X-cc', 'X-bcc', 'Subject', 'Message']
df = pd.DataFrame(index=[], columns=cols)

In [101]:
message = mail.get_payload()

In [119]:
mail.keys()

['Message-ID',
 'Date',
 'From',
 'Subject',
 'Cc',
 'Mime-Version',
 'Content-Type',
 'Content-Transfer-Encoding',
 'Bcc',
 'X-From',
 'X-To',
 'X-cc',
 'X-bcc',
 'X-Folder',
 'X-Origin',
 'X-FileName']

In [121]:
mail.get('X-cc')

'Hayden, Frank </O=ENRON/OU=NA/CN=RECIPIENTS/CN=Fhayden>, Gossett, Jeffrey C. </O=ENRON/OU=NA/CN=RECIPIENTS/CN=Jgosset>'